# Secondary standard stars

In [32]:
import astropy
from astropy.io import fits
from astropy.stats import sigma_clipped_stats
from astropy.coordinates import SkyCoord, match_coordinates_sky
from astropy import units as u
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from photutils.aperture import CircularAperture, CircularAnnulus, aperture_photometry
from astropy.wcs import WCS
import warnings
from astropy.wcs import FITSFixedWarning

warnings.filterwarnings('ignore', category=FITSFixedWarning, message=".*'datfix' made the change.*")

wd = r"C:\Users\friesco\workstation\fr-p\studies\ASTRO716\data_excercise"
os.chdir(wd)
plt.rcParams['figure.figsize'] = [10, 8]

calib_df = pd.read_csv('calibration_stars.csv')
files = sorted([f for f in os.listdir(wd) if f.startswith('phot_') and f.endswith('.fits')])

try:
    ps1_cat = pd.read_csv('psdr1_new.tsv', sep='\t')
    ps1_cat.loc[ps1_cat['RAJ2000'] == '-------', 'RAJ2000'] = np.nan
    ps1_cat.loc[ps1_cat['DEJ2000'] == '-------', 'DEJ2000'] = np.nan
    for col in ['RAJ2000', 'DEJ2000', 'rmag', 'e_rmag', 'gmag']:
        ps1_cat[col] = pd.to_numeric(ps1_cat[col], errors='coerce')
    ps1_cat.dropna(subset=['RAJ2000', 'DEJ2000', 'rmag', 'e_rmag'], inplace=True)
    ps1_coords = SkyCoord(ra=ps1_cat['RAJ2000'], dec=ps1_cat['DEJ2000'], unit="deg")
except Exception as e:
    print(f"Error loading PS1 catalog: {e}")
    ps1_coords = None

def phot_cts(img, x, y, ap_r, in_r, out_r):
    ap = CircularAperture((x, y), r=ap_r)
    ann = CircularAnnulus((x, y), r_in=in_r, r_out=out_r)
    phot = aperture_photometry(img, [ap, ann])
    ann_msk = ann.to_mask(method='center')
    bkg_vals = ann_msk.multiply(img)[ann_msk.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    return phot['aperture_sum_0'][0], med * ap.area, ap.area

def calc_snr(cts, bkg_cts, ap_area):
    mean_cts = (cts - bkg_cts) / ap_area
    if cts + bkg_cts <= 0:
        return 0.0
    err_cts = np.sqrt(cts + bkg_cts) / ap_area
    return mean_cts / err_cts

def phot_mag(img, x, y, ap_r, in_r, out_r):
    ap = CircularAperture((x, y), r=ap_r)
    ann = CircularAnnulus((x, y), r_in=in_r, r_out=out_r)
    phot = aperture_photometry(img, [ap, ann])
    ann_msk = ann.to_mask(method='center')
    bkg_vals = ann_msk.multiply(img)[ann_msk.data > 0]
    mean, med, std = sigma_clipped_stats(bkg_vals, sigma=3.0)
    bkg_cts = med * ap.area
    final_cts = phot['aperture_sum_0'] - bkg_cts
    if final_cts[0] > 0:
        mag = -2.5 * np.log10(final_cts) + 50
        mag_err = 1.0857 * np.sqrt(phot['aperture_sum_0'] + bkg_cts) / phot['aperture_sum_0']
    else:
        mag, mag_err = np.nan, np.nan
    return mag, mag_err, med

nx, ny = 1033, 1336
ap_rad_snr = np.arange(5, 40, 1)
in_r, out_r = 25, 35
opt_ap = []

for i, f in enumerate(files):
    path = os.path.join(wd, f)
    with fits.open(path) as h:
        img = h[0].data
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    snrs = []
    for ap_r in ap_rad_snr:
        tot_cts, bkg_cts, ap_area = phot_cts(img, nx, ny, ap_r, in_r, out_r)
        snr = calc_snr(tot_cts, bkg_cts, ap_area) if tot_cts + bkg_cts > 0 else 0.0
        snrs.append(snr)

    opt_ap.append(ap_rad_snr[np.argmax(snrs)])

if ps1_coords:
    calib_df.reset_index(drop=True, inplace=True)
    calib_coords = SkyCoord(ra=calib_df['RAJ2000'], dec=calib_df['DEJ2000'], unit="deg")
    idx, d2d, d3d = match_coordinates_sky(calib_coords, ps1_coords)
    good = (ps1_cat['rmag'].iloc[idx] >= 14) & (ps1_cat['rmag'].iloc[idx] <= 19) & (ps1_cat['e_rmag'].iloc[idx] <= 0.1)
    match_idx = np.where(good)[0]
    ps1_match = ps1_cat.iloc[idx[good]]
    calib_match = calib_df.iloc[match_idx]
else:
    ps1_match = pd.DataFrame()
    calib_match = pd.DataFrame()

results = []
for i, f in enumerate(files):
    path = os.path.join(wd, f)
    with fits.open(path) as h:
        img = h[0].data
        hdr = h[0].header
        w = WCS(hdr)
        if img.dtype.byteorder == '>':
            img = img.byteswap().view(img.dtype.newbyteorder('<'))

    opt_r = opt_ap[i]
    mag_nova, err_nova, bkg_nova = phot_mag(img, nx, ny, opt_r, in_r, out_r)
    ra_nova, dec_nova = w.pixel_to_world_values(nx, ny)

    for idx, ps1_star in ps1_match.iterrows():
        x, y = w.world_to_pixel(SkyCoord(ps1_star['RAJ2000'], ps1_star['DEJ2000'], unit="deg"))
        mag, err, bkg = phot_mag(img, x, y, opt_r, in_r, out_r)
        r_prime = ps1_star['rmag'] - 0.001 + 0.011 * (ps1_star['gmag'] - ps1_star['rmag'])
        r_prime_err = 0.004
        results.append({
            'img': f, 'mjd': hdr['MJD'], 'type': 'nova' if idx == -1 else 'ps1_star',
            'id': 'nova' if idx == -1 else ps1_star['ID'], 'ra': ra_nova if idx == -1 else ps1_star['RAJ2000'],
            'dec': dec_nova if idx == -1 else ps1_star['DEJ2000'], 'x': nx if idx == -1 else x, 'y': ny if idx == -1 else y,
            'ap_r': opt_r, 'mag': mag_nova if idx == -1 else mag, 'mag_err': err_nova if idx == -1 else err,
            'bkg': bkg_nova if idx == -1 else bkg, 'r_prime': np.nan if idx == -1 else r_prime, 'r_prime_err': np.nan if idx == -1 else r_prime_err
        })

results_df = pd.DataFrame(results)
results_df.to_csv('photometry_results.csv', index=False)
